This project is based on kaggle's project

[Blue Book for Bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers)

Obejctive - Predict the auction sale price for a piece of heavy equipment to create a "blue book" for bulldozers

We're going to take the following approach:  

1. Problem definition
2. Data  
    Train and Valid are combined and in TrainAndValid. Both of them have price column       
3. Evaluation
4. Features
5. Modelling
6. Experimentation


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import datetime

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import classification_report

In [2]:
data_dictionary = pd.read_excel("Data Dictionary.xlsx")
#to shorten the description and read more
data_dictionary["Description"] = data_dictionary["Description"].str.replace("machine configuration", "MC")
# data_dictionary["Description"] = data_dictionary["Description"] + data_dictionary["Unnamed: 2"].astype(str)
# data_dictionary.drop("Unnamed: 2", axis = 1, inplace = True)
data_dictionary

,Variable,Description,Unnamed: 2
0,SalesID,unique identifier of a particular sale of a ...,NaN
1,MachineID,identifier for a particular machine; machin...,NaN
2,ModelID,identifier for a unique machine model (i.e. ...,NaN
3,datasource,source of the sale record; some sources are...,NaN
4,auctioneerID,"identifier of a particular auctioneer, i.e. ...",NaN
5,YearMade,year of manufacturer of the Machine,NaN
6,MachineHoursCurrentMeter,current usage of the machine in hours at tim...,NaN
7,UsageBand,"value (low, medium, high) calculated compari...",NaN
8,Saledate,time of sale,NaN
9,Saleprice,cost of sale in USD,NaN


In [3]:
validSolution = pd.read_csv("ValidSolution.csv")
validSolution.head()

,SalesID,SalePrice,Usage
0,1222837,31000.0,PublicTest
1,1222839,54000.0,PublicTest
2,1222841,26500.0,PublicTest
3,1222843,10000.0,PublicTest
4,1222845,35000.0,PublicTest


In [4]:
validSolution["Usage"].value_counts()

## this means that Usage for all the product is same only, in other words, Usage is not really useful and we 
## we can omit it in our final value

PublicTest    11573
Name: Usage, dtype: int64

In [5]:
validSolution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SalesID    11573 non-null  int64  
 1   SalePrice  11573 non-null  float64
 2   Usage      11573 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 271.4+ KB


In [6]:
valid = pd.read_csv("Valid.csv")
valid.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
1,1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,...,None or Unspecified,"12' 4""",None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN
2,1222841,999308,4587,121,3,2000,10127.0,Medium,1/5/2012 0:00,270LC,...,None or Unspecified,"12' 4""",None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1222843,1062425,1954,121,3,1000,4682.0,Low,1/5/2012 0:00,892DLC,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
4,1222845,1032841,4701,121,3,2002,8150.0,Medium,1/4/2012 0:00,544H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [7]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 4031 non-null   object 
 8   saledate                  11573 non-null  object 
 9   fiModelDesc               11573 non-null  object 
 10  fiBaseModel               11573 non-null  object 
 11  fiSecondaryDesc           8037 non-null   object 
 12  fiModelSeries             1759 non-null   object 
 13  fiModelDescriptor         2897 non-null   object 
 14  Produc

In [8]:
#TrainAndValid is the combination of train and valid (2 seperate csv)
trainAndValid = pd.read_csv("TrainAndValid.csv")
trainAndValid.head()

C:\Users\Harshal Garg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (13,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
train = pd.read_csv("train.csv")
train.head()

C:\Users\Harshal Garg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (13,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Splitting the data in X and y
train_X = train.drop("SalePrice", axis = 1)
train_y = train["SalePrice"]

In [11]:
trainAndValid.shape, trainAndValid.isnull().sum()

((412698, 53),
 SalesID                          0
 SalePrice                        0
 MachineID                        0
 ModelID                          0
 datasource                       0
 auctioneerID                 20136
 YearMade                         0
 MachineHoursCurrentMeter    265194
 UsageBand                   339028
 saledate                         0
 fiModelDesc                      0
 fiBaseModel                      0
 fiSecondaryDesc             140727
 fiModelSeries               354031
 fiModelDescriptor           337882
 ProductSize                 216605
 fiProductClassDesc               0
 state                            0
 ProductGroup                     0
 ProductGroupDesc                 0
 Drive_System                305611
 Enclosure                      334
 Forks                       214983
 Pad_Type                    331602
 Ride_Control                259970
 Stick                       331602
 Transmission                224691
 Turbocharged

In [12]:
#there is not point to see the test data as it will be same as train and valid
test = pd.read_csv("Test.csv")
test

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829,1006309,3168,121,3,1999,3688.0,Low,5/3/2012 0:00,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844,1022817,7271,121,3,1000,28555.0,High,5/10/2012 0:00,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847,1031560,22805,121,3,2004,6038.0,Medium,5/10/2012 0:00,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848,56204,1269,121,3,2006,8940.0,High,5/10/2012 0:00,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863,1053887,22312,121,3,2005,2286.0,Low,5/10/2012 0:00,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12452,6643171,2558317,21450,149,2,2008,NaN,NaN,10/24/2012 0:00,80NX3,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
12453,6643173,2558332,21434,149,2,2005,NaN,NaN,10/24/2012 0:00,28N,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
12454,6643184,2558342,21437,149,2,1000,NaN,NaN,10/24/2012 0:00,35N,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
12455,6643186,2558343,21437,149,2,2006,NaN,NaN,10/24/2012 0:00,35N,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [13]:
# test_null = pd.DataFrame(test.info())
# test_null
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   12457 non-null  int64  
 1   MachineID                 12457 non-null  int64  
 2   ModelID                   12457 non-null  int64  
 3   datasource                12457 non-null  int64  
 4   auctioneerID              12457 non-null  int64  
 5   YearMade                  12457 non-null  int64  
 6   MachineHoursCurrentMeter  2129 non-null   float64
 7   UsageBand                 1834 non-null   object 
 8   saledate                  12457 non-null  object 
 9   fiModelDesc               12457 non-null  object 
 10  fiBaseModel               12457 non-null  object 
 11  fiSecondaryDesc           8482 non-null   object 
 12  fiModelSeries             2006 non-null   object 
 13  fiModelDescriptor         3024 non-null   object 
 14  Produc

In [14]:
rfbt = pd.read_csv("random_forest_benchmark_test.csv")
rfbt.head()

,SalesID,SalePrice
0,1227829,13630.0
1,1227844,32530.0
2,1227847,48660.0
3,1227848,96080.0
4,1227863,27440.0


In [15]:
rfbt.shape

(12457, 2)

In [16]:
median_benchmark = pd.read_csv("median_benchmark.csv")
median_benchmark.head()


,SalesID,SalePrice
0,1222837,24000.0
1,1222839,24000.0
2,1222841,24000.0
3,1222843,24000.0
4,1222845,24000.0


In [17]:
machine_appendix = pd.read_csv("machine_appendix.csv")
machine_appendix.head()

,MachineID,ModelID,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,fiProductClassDesc,ProductGroup,ProductGroupDesc,MfgYear,fiManufacturerID,fiManufacturerDesc,PrimarySizeBasis,PrimaryLower,PrimaryUpper
0,113,1355,350L,350,NaN,NaN,L,"Hydraulic Excavator, Track - 50.0 to 66.0 Metr...",TEX,Track Excavators,1994.0,26,Caterpillar,Weight - Metric Tons,50.0,66.0
1,434,3538,416C,416,C,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,BL,Backhoe Loaders,1997.0,26,Caterpillar,Standard Digging Depth - Ft,14.0,15.0
2,534,3538,416C,416,C,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,BL,Backhoe Loaders,1998.0,26,Caterpillar,Standard Digging Depth - Ft,14.0,15.0
3,718,3538,416C,416,C,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,BL,Backhoe Loaders,2000.0,26,Caterpillar,Standard Digging Depth - Ft,14.0,15.0
4,1753,1580,D5GLGP,D5,G,NaN,LGP,"Track Type Tractor, Dozer - 85.0 to 105.0 Hors...",TTT,Track Type Tractors,2006.0,26,Caterpillar,Horsepower,85.0,105.0


In [18]:
machine_appendix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358593 entries, 0 to 358592
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   MachineID           358593 non-null  int64  
 1   ModelID             358593 non-null  int64  
 2   fiModelDesc         358593 non-null  object 
 3   fiBaseModel         358593 non-null  object 
 4   fiSecondaryDesc     236838 non-null  object 
 5   fiModelSeries       47171 non-null   object 
 6   fiModelDescriptor   64534 non-null   object 
 7   fiProductClassDesc  358593 non-null  object 
 8   ProductGroup        358593 non-null  object 
 9   ProductGroupDesc    358593 non-null  object 
 10  MfgYear             358361 non-null  float64
 11  fiManufacturerID    358593 non-null  int64  
 12  fiManufacturerDesc  358593 non-null  object 
 13  PrimarySizeBasis    353801 non-null  object 
 14  PrimaryLower        353801 non-null  float64
 15  PrimaryUpper        353801 non-nul

In [19]:
trainAndValid.shape,\
train_X.shape, train_y.shape, \
valid.shape, validSolution.shape, median_benchmark.shape,\
test.shape, rfbt.shape, \
machine_appendix.shape

((412698, 53),
 (401125, 52),
 (401125,),
 (11573, 52),
 (11573, 3),
 (11573, 2),
 (12457, 52),
 (12457, 2),
 (358593, 16))

In [20]:
#slicing the time part and converting in datetime format
# from datetime import datetime

def slice_and_convert(df):
    temp = df["saledate"].str.slice(stop = -5) #extracting th date


    df["saledate"] = pd.to_datetime(temp, 
                          format = "%m/%d/%y", #format
                          infer_datetime_format=True) #deal with multiple format
    

In [21]:
#converting into datetime

slice_and_convert(train_X)
slice_and_convert(valid)
slice_and_convert(test)
slice_and_convert(train)

In [22]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 52 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   401125 non-null  int64         
 1   MachineID                 401125 non-null  int64         
 2   ModelID                   401125 non-null  int64         
 3   datasource                401125 non-null  int64         
 4   auctioneerID              380989 non-null  float64       
 5   YearMade                  401125 non-null  int64         
 6   MachineHoursCurrentMeter  142765 non-null  float64       
 7   UsageBand                 69639 non-null   object        
 8   saledate                  401125 non-null  datetime64[ns]
 9   fiModelDesc               401125 non-null  object        
 10  fiBaseModel               401125 non-null  object        
 11  fiSecondaryDesc           263934 non-null  object        
 12  fi

In [23]:
# np.random.seed(42)

# fields_categorical = [  "fiProductClassDesc", "state", "fiBaseModel", 
#                               "ProductGroup", "ProductGroupDesc", 
#                               "Enclosure", "Hydraulics"]

# fields_numercial = ["SalesID", "MachineID", 
#                              "ModelID", "datasource", 
#                              "auctioneerID", "YearMade"]

# categorical_transformer = Pipeline(steps = [
#     ("imputer", SimpleImputer(strategy = 'constant', fill_value = 'missing')),
#     ("OrdinalEncoder", OrdinalEncoder())])

# numeric_transform = Pipeline(steps = [
#     ("imputer", SimpleImputer(strategy = "median"))
# ])

# preprocessor = ColumnTransformer(transformers = [
#     ("cat", categorical_transformer, fields_categorical),
#     ("num", numeric_transform, fields_numercial)
# ])

# model = Pipeline(steps = [("preprocessor", preprocessor),
#                          ("model", LogisticRegression(max_iter = 1000))])

# # Taking a sample test set

# #we need to take a random sample with result so the order doesn't disturb


# train_sample = train.sample(frac = 0.02)

# train_sample_X = train_sample[fields_categorical + fields_numercial]
# train_sample_y = train_sample["SalePrice"]

# sample_train_X, sample_test_X, \
# sample_train_y, sample_test_y = train_test_split(train_sample_X, train_sample_y, test_size = 0.2)

# model.fit(sample_train_X, sample_train_y)
# model.score(sample_test_X, sample_test_y)

In [24]:
# np.random.seed(42)

# train_sample = train.sample(frac = 0.02)
# train_sample = train_sample[["fiBaseModel", "fiProductClassDesc"]]

# enc = OrdinalEncoder()
# enc.fit(train_sample)
# enc.categories_

In [25]:
#creating a sample which will be preprocessed to fit a model

np.random.seed(42)

train_sample = train.sample(frac = 1) #by changing this number you can choose ratio of the sample to train

fields_numercial = ["SalesID", "MachineID", 
                             "ModelID", "datasource", 
                             "auctioneerID", "YearMade"]
#Imputing the numeric data
num_trans = SimpleImputer(strategy='median')
train_sample[fields_numercial] = num_trans.fit_transform(train_sample[fields_numercial])
# num_trans.transform([fields_numerical])

fields_categorical = ["fiProductClassDesc", "state", "fiBaseModel", 
                      "ProductGroup", "Hydraulics",
                      "ProductGroupDesc", "Enclosure"]

# #Imputing the categorical data
cat_trans = SimpleImputer(strategy='constant')
train_sample[fields_categorical] = cat_trans.fit_transform(train_sample[fields_categorical])
# cat_trans.transform([fields_categorical])

cat_enc = OrdinalEncoder()
train_sample[fields_categorical] = cat_enc.fit_transform(train_sample[fields_categorical])

In [26]:
#check the transformed data 
train_sample[fields_categorical].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401125 entries, 347851 to 121958
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   fiProductClassDesc  401125 non-null  float64
 1   state               401125 non-null  float64
 2   fiBaseModel         401125 non-null  float64
 3   ProductGroup        401125 non-null  float64
 4   Hydraulics          401125 non-null  float64
 5   ProductGroupDesc    401125 non-null  float64
 6   Enclosure           401125 non-null  float64
dtypes: float64(7)
memory usage: 24.5 MB


Now that we have transformed the data in the format suitable for training, we can split it in X, y and test and train

In [27]:
# np.random.seed(42)

# train_sample_X = train_sample[fields_categorical + fields_numercial]
# train_sample_y = train_sample["SalePrice"]

# # not requied in RandomSearchCV
# # sample_train_X, sample_test_X, \
# # sample_train_y, sample_test_y = train_test_split(train_sample_X, train_sample_y, 
# #                                                  test_size = 0.2)  
# # Number of trees in random forest

# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']

# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)

# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]

# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]

# # Method of selecting samples for training each tree
# bootstrap = [True, False]# Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [28]:
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# model = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = model, 
#                                param_distributions = random_grid, 
#                                n_iter = 30, cv = 3, verbose=2, 
#                                random_state=42, n_jobs = -2)

# rf_random.fit(train_sample_X, train_sample_y)

In [ ]:
# rf_random.best_params_


In [ ]:
# model.score(sample_test_X, sample_test_y)

With 2% data, accuracy was 69%  
With 20% data, accuracy was 75.94%

In [ ]:
# best_parameter = {'n_estimators': 1200,
#  'min_samples_split': 2,
#  'min_samples_leaf': 2,
#  'max_features': 'sqrt',
#  'max_depth': 50,
#  'bootstrap': False}

np.random.seed(42)

train_sample_X = train_sample[fields_categorical + fields_numercial]
train_sample_y = train_sample["SalePrice"]

# not requied in RandomSearchCV
sample_train_X, sample_test_X, \
sample_train_y, sample_test_y = train_test_split(train_sample_X, train_sample_y, 
                                                 test_size = 0.2)  
model_2 = RandomForestRegressor(n_estimators = 1200, min_samples_split = 2, min_samples_leaf = 2, 
                                max_features = 'sqrt', max_depth = 50, bootstrap =  False, 
                                n_jobs = -2)

model_2.fit(sample_train_X, sample_train_y)
model_2.score(sample_test_X, sample_test_y)

In [ ]:
temp = pd.DataFrame(data = model.feature_importances_, index = sample_train_X.columns, )
temp.plot.bar();

In [ ]:
model.fit(sample_train_X.drop(["datasource", "Hydraulics"], axis = 1), sample_train_y)
model.score(sample_test_X.drop(["datasource", "Hydraulics"], axis = 1), sample_test_y)

Without "datasource" and "Hydraulics", accuracy was 76.05%

In [ ]:
# print(model.classification_report)

to reduce the number of features we will use `np.where`

After hyperparameter tunning and 2% data - 71%  
After hyperparameter tunning and 20% data - 77.93  
After hyperparameter tunning and 100% data - 
